In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import svm
import pickle

In [2]:
data = pd.read_csv("2_500_z2_new.csv",error_bad_lines=False)
data

,mean,sum,minimum,maximum,std,var,kurtosis,skewness,condition
0,0.528640,0.535890,-0.38271,1.20660,0.15396,0.023705,4.0432,-0.38793,1
1,0.532720,0.540350,-0.35960,1.06760,0.15213,0.023143,3.9499,-0.40600,1
2,0.531500,0.536060,-0.20506,1.12870,0.15375,0.023639,3.6487,-0.27011,1
3,0.508890,0.517570,-0.40095,1.00740,0.15433,0.023816,4.0031,-0.44341,1
4,0.526380,0.534950,-0.55289,1.38910,0.15438,0.023833,4.4268,-0.37518,1
...,...,...,...,...,...,...,...,...,...
1595,0.040631,0.043709,-0.55790,0.48150,0.10493,0.011011,3.8001,-0.26962,16
1596,0.042683,0.044624,-0.74374,0.98350,0.10524,0.011076,5.0817,-0.11603,16
1597,0.043313,0.045870,-0.73517,0.69698,0.10842,0.011754,4.4859,-0.29430,16
1598,0.043133,0.045828,-0.70864,0.57921,0.10738,0.011531,3.9885,-0.20763,16


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   mean       1600 non-null   float64
 1   sum        1600 non-null   float64
 2   minimum    1600 non-null   float64
 3   maximum    1600 non-null   float64
 4   std        1600 non-null   float64
 5   var        1600 non-null   float64
 6   kurtosis   1600 non-null   float64
 7   skewness   1600 non-null   float64
 8   condition  1600 non-null   int64  
dtypes: float64(8), int64(1)
memory usage: 112.6 KB


In [4]:
data=data.drop(columns = ['maximum','minimum'])
data

,mean,sum,std,var,kurtosis,skewness,condition
0,0.528640,0.535890,0.15396,0.023705,4.0432,-0.38793,1
1,0.532720,0.540350,0.15213,0.023143,3.9499,-0.40600,1
2,0.531500,0.536060,0.15375,0.023639,3.6487,-0.27011,1
3,0.508890,0.517570,0.15433,0.023816,4.0031,-0.44341,1
4,0.526380,0.534950,0.15438,0.023833,4.4268,-0.37518,1
...,...,...,...,...,...,...,...
1595,0.040631,0.043709,0.10493,0.011011,3.8001,-0.26962,16
1596,0.042683,0.044624,0.10524,0.011076,5.0817,-0.11603,16
1597,0.043313,0.045870,0.10842,0.011754,4.4859,-0.29430,16
1598,0.043133,0.045828,0.10738,0.011531,3.9885,-0.20763,16


In [5]:
data['condition'] = pd.Categorical(data['condition'])

In [6]:
train_data, test_data = train_test_split(data, test_size = 0.3, stratify = data['condition'],
                                         random_state = 1234)
test_data['condition'].value_counts()

1     30
2     30
3     30
4     30
5     30
6     30
7     30
8     30
9     30
10    30
11    30
12    30
13    30
14    30
15    30
16    30
Name: condition, dtype: int64

In [7]:
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(train_data.iloc[:,:-1])

In [8]:
test_data_scaled = (test_data.iloc[:,:-1].values - scaler.mean_)/np.sqrt(scaler.var_)

In [9]:
parameters = {"C":[1, 10, 50, 100, 300, 500],
             'gamma':[0.01, 0.05, 0.1, 0.5, 1, 5],
             'kernel':["rbf"]}

In [10]:
tuned_svm_clf = GridSearchCV(SVC(),parameters,n_jobs = -1, cv= 10)

In [11]:
tuned_svm_clf.fit(train_data_scaled, train_data['condition'])

GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [1, 10, 50, 100, 300, 500],
                         'gamma': [0.01, 0.05, 0.1, 0.5, 1, 5],
                         'kernel': ['rbf']})

In [12]:
tuned_svm_clf.best_score_

0.9321428571428573

In [13]:
tuned_svm_clf.best_params_

{'C': 500, 'gamma': 0.05, 'kernel': 'rbf'}

In [14]:
best_clf = tuned_svm_clf.best_estimator_
best_clf

SVC(C=500, gamma=0.05)

In [15]:
train_predictions = best_clf.predict(train_data_scaled)
test_predictions = best_clf.predict(test_data_scaled)

In [16]:
filename = 'finalized_model3.pkl'
pickle.dump(best_clf, open(filename, 'wb'))